In [2]:
# install requirements (one-time)
#!pip install google-cloud-bigquery google-cloud-storage tqdm
!pip install google-auth

## Read Configurations

## Define Imports and Functions

In [23]:
import os
import random
import string
import time
from typing import List, Optional, Tuple

import pandas as pd

from google.oauth2 import service_account
from google.cloud import bigquery
from google.cloud import storage
from google.api_core.exceptions import NotFound, Forbidden, GoogleAPICallError
from tqdm import tqdm
from google.api_core.exceptions import Conflict

In [4]:
# Creates new name with 2 number suffix
def _make_random_suffix(n=2):
    return ''.join(random.choices('0123456789', k=n))

In [5]:
def _get_source_client_from_key(key_path: str) -> bigquery.Client:
    creds = service_account.Credentials.from_service_account_file(key_path)
    return bigquery.Client(credentials=creds, project=creds.project_id)

In [6]:
src_creds = service_account.Credentials.from_service_account_file("../secrets/darry-r26.json")

In [7]:
src_client = bigquery.Client(credentials=src_creds, project=src_creds.project_id)

In [8]:
dataset_names = []
for datasets in src_client.list_datasets():
    # save under a plan
    dataset_names.append(f'{datasets.project}.{datasets.dataset_id}')

In [9]:
len(dataset_names)

34

In [10]:
for projects in src_client.list_projects():
    print(projects.project_id)

data-analytics-ns


In [11]:
# for dataset in dataset_names:
for table in src_client.list_tables('data-analytics-ns.course15'):
    print(table.full_table_id)

data-analytics-ns:course15.cc_funnel
data-analytics-ns:course15.cc_funnel_kpi
data-analytics-ns:course15.cc_funnel_local
data-analytics-ns:course15.cc_parcel
data-analytics-ns:course15.cc_parcel_kpi
data-analytics-ns:course15.cc_parcel_kpi_formatted
data-analytics-ns:course15.cc_parcel_kpi_global
data-analytics-ns:course15.cc_parcel_kpi_month
data-analytics-ns:course15.cc_parcel_kpi_priority
data-analytics-ns:course15.cc_parcel_kpi_priority_by_ratio
data-analytics-ns:course15.cc_parcel_kpi_transporter
data-analytics-ns:course15.cc_parcel_product
data-analytics-ns:course15.circle_sales
data-analytics-ns:course15.circle_sales_daily
data-analytics-ns:course15.circle_stock
data-analytics-ns:course15.circle_stock_cat
data-analytics-ns:course15.circle_stock_copy
data-analytics-ns:course15.circle_stock_d7_ch2
data-analytics-ns:course15.circle_stock_kpi
data-analytics-ns:course15.circle_stock_kpi_top
data-analytics-ns:course15.circle_stock_model_type
data-analytics-ns:course15.circle_stock_nam

In [12]:
src_client.query_and_wait('select * from data-analytics-ns.course15.cc_parcel_kpi_global')

## Destination Client Initialization

In [16]:
dst_creds = service_account.Credentials.from_service_account_file("../secrets/storiesmitdee.json")
dst_client = bigquery.Client(credentials=dst_creds, project=dst_creds.project_id)

# Define dataset
dataset_name = f'{dst_client.project}.test01_db'
dataset = bigquery.Dataset(dataset_name)
dataset.location = "EU"  # required, matches your migration region

# Create dataset (if not exists)
try:
    dataset = dst_client.create_dataset(dataset)  # API request
    print(f"✅ Created dataset {dataset_name}")
except Exception as e:
    print(f"⚠️ Dataset {dataset_name} already exists")

✅ Created dataset data-analytics-ns-470609.test01_db


In [21]:
src_table_id = f"{src_client.project}.course15.cc_funnel"
dst_table_id = f"{dst_client.project}.test01_db.my_table_copy"

# Configure the copy job
job = dst_client.copy_table(src_table_id, dst_table_id, location="EU")

# Wait for job to complete
job.result()
print(f"✅ Copied table {src_table_id} → {dst_table_id}")

BadRequest: 400 data-analytics-ns:course15.cc_funnel is not allowed for this operation because it is currently a EXTERNAL.; reason: invalid, message: data-analytics-ns:course15.cc_funnel is not allowed for this operation because it is currently a EXTERNAL.

In [20]:
rows = src_client.list_rows(src_table_id)
df = rows.to_dataframe()   # Pandas dataframe with data

ValueError: Please install the 'db-dtypes' package to use this function.

In [40]:
# Source & destination
src_table_id = f"{src_client.project}.course15.cc_funnel_native"
dst_table_id = f"{dst_client.project}.test01_db.cc_funnel_copy"

# 1️⃣ Read source table into DataFrame
try:
    df = src_client.query(f"SELECT * FROM `{src_table_id}`").to_dataframe()
except:
    src_client.query(f"CREATE OR REPLACE TABLE `{src_table_id}_native` AS SELECT * FROM `{src_table_id}`")
    df = src_client.query(f"SELECT * FROM `{src_table_id}_native`").to_dataframe()


print(f"✅ Read {len(df)} rows from source table")

# 2️⃣ Load DataFrame into destination table
job = dst_client.load_table_from_dataframe(df, dst_table_id)
job.result()
print(f"✅ Loaded {len(df)} rows into destination table {dst_table_id}")

NotFound: 404 Not found: Table data-analytics-ns:course15.cc_funnel_native_native was not found in location EU; reason: notFound, message: Not found: Table data-analytics-ns:course15.cc_funnel_native_native was not found in location EU

Location: EU
Job ID: 84b3c6c0-389b-4c29-bc42-f448bbdb2a65


In [25]:
df = src_client.query(f"SELECT * FROM `{src_table_id}`").to_dataframe()

Forbidden: 403 Access Denied: BigQuery BigQuery: Permission denied while getting Drive credentials.; reason: accessDenied, location: 1YJOR3XYHU3-KaxvI95bDgmVSeUjDXrEsTJnbOGYftE8, message: Access Denied: BigQuery BigQuery: Permission denied while getting Drive credentials.

Location: EU
Job ID: cd923b73-17ce-4396-9a43-6e1fed8e9413


ModuleNotFoundError: No module named 'google.colab'

In [29]:
native_table_id = f"{src_client.project}.course15.cc_funnel_native"  # underscore

try:
    # Try to read the table (external table)
    df = src_client.query(f"SELECT * FROM `{src_table_id}`").to_dataframe()
except Exception as e:
    print("External table blocked, creating native copy...", e)
    
    # Create a native table in the same dataset
    create_job = src_client.query(
        f"""
        CREATE OR REPLACE TABLE `{native_table_id}` AS
        SELECT * FROM `{src_table_id}`
        """
    )
    create_job.result()  # wait for completion

    # Now read from native table
    df = src_client.query(f"SELECT * FROM `{native_table_id}`").to_dataframe()


External table blocked, creating native copy... 403 Access Denied: BigQuery BigQuery: Permission denied while getting Drive credentials.; reason: accessDenied, location: 1YJOR3XYHU3-KaxvI95bDgmVSeUjDXrEsTJnbOGYftE8, message: Access Denied: BigQuery BigQuery: Permission denied while getting Drive credentials.

Location: EU
Job ID: f2e48d2e-3a0a-4a84-88dd-bcbc570e292e



Forbidden: 403 Access Denied: BigQuery BigQuery: Permission denied while getting Drive credentials.; reason: accessDenied, location: 1YJOR3XYHU3-KaxvI95bDgmVSeUjDXrEsTJnbOGYftE8, message: Access Denied: BigQuery BigQuery: Permission denied while getting Drive credentials.

Location: EU
Job ID: 9c8c2df8-ae4c-4a2d-a4d6-3f35f32d9540


In [38]:
import pandas as pd

# Make sure you have the library
# pip install pandas-gbq --upgrade

# Fully-qualified source table
src_table_id = f"{src_client.project}.course15.cc_funnel_native"

# Read table into DataFrame
df = pd.read_gbq(f"SELECT * FROM `{src_table_id}`",
                 project_id=src_client.project,
                 credentials=src_client._credentials)  # pass source credentials
print(f"✅ Read {len(df)} rows from source table")

/var/folders/1b/n00h51m115q53l5qwk1gc0m40000gn/T/ipykernel_49881/630193174.py:10: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df = pd.read_gbq(f"SELECT * FROM `{src_table_id}`",


ValueError: Please install the 'db-dtypes' package to use this function.

In [37]:
import db_dtypes

In [41]:
from google.cloud import bigquery
from google.api_core.exceptions import Conflict

# ---------- Destination client ----------
dst_client = bigquery.Client(credentials=dst_creds, project=dst_creds.project_id)

# ---------- Table IDs ----------
src_table_id = "data-analytics-ns.course15.cc_parcel_kpi_formatted"  # fully-qualified source table
dst_dataset_id = f"{dst_client.project}.test01_db"            # destination dataset
dst_table_name = "source_table_copy"                          # destination table name
dst_table_id = f"{dst_dataset_id}.{dst_table_name}"

# ---------- Create destination dataset if it doesn't exist ----------
try:
    dataset = bigquery.Dataset(dst_dataset_id)
    dataset.location = "EU"
    dst_client.create_dataset(dataset)
    print(f"✅ Created dataset {dst_dataset_id}")
except Conflict:
    print(f"⚠️ Dataset {dst_dataset_id} already exists")

# ---------- Copy table ----------
from google.cloud.bigquery import CopyJobConfig

job_config = CopyJobConfig(
    write_disposition="WRITE_TRUNCATE"  # overwrite if table exists
)

copy_job = dst_client.copy_table(
    sources=src_table_id,       # source table (fully-qualified)
    destination=dst_table_id,   # destination table
    location="EU",              # dataset location
    job_config=job_config
)

copy_job.result()  # wait for completion
print(f"✅ Copied {src_table_id} → {dst_table_id}")


⚠️ Dataset data-analytics-ns-470609.test01_db already exists
✅ Copied data-analytics-ns.course15.cc_parcel_kpi_formatted → data-analytics-ns-470609.test01_db.source_table_copy
